LangGraph has a built-in persistence layer, implemented through checkpointers. When you compile a graph with a checkpointer, the checkpointer saves a `checkpoint` of the graph state at every super-step. Those checkpoints are saved to a `thread`, which can be accessed after graph execution.

# Threads

A thread is a unique ID or thread identifier assigned to each checkpoint saved by a checkpointer.

When invoking a graph with a checkpointer, you must specify a thread_id as part of the configurable portion of the config.
```Python
{"configurable": {"thread_id": "1"}}
```

# Checkpoints

The state of a thread at a particular point in time is called a checkpoint. Checkpoint is a snapshot of the graph state saved at each super-step and is represented by `StateSnapshot` object.

In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

## Get state

View the latest state of the graph

In [5]:
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-6952-6754-8002-fb1105b5d78b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-04T08:28:17.249053+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-6951-6020-8001-4702b2ed714f'}}, tasks=(), interrupts=())

## Get state history

get the full history of the graph execution for a given thread

In [4]:
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-6952-6754-8002-fb1105b5d78b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-04T08:28:17.249053+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-6951-6020-8001-4702b2ed714f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-6951-6020-8001-4702b2ed714f'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-04T08:28:17.248458+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0b9583-694e-624e-8000-5cb03659d424'}}, tasks=(PregelTask(id='0a384303-18ae-1496-7fb7-a92945ad1f8a', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

## Replay

If we invoke a graph with a `thread_id` and a `checkpoint_id`, then we will re-play the previously executed steps before a checkpoint that corresponds to the checkpoint_id, and only execute the steps after the checkpoint.

In [7]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f0b9583-694e-624e-8000-5cb03659d424"}}
graph.invoke(None, config=config)

{'foo': 'b', 'bar': ['a', 'b']}

## Update state

# Memory Store

Consider the case of a chatbot where we want to retain specific information about the user across all chat conversations (e.g., threads) with that user.

With checkpointers alone, we cannot share information across threads. This motivates the need for the `Store` interface. As an illustration, we can define an `InMemoryStore` to store information about a user across threads. We simply compile our graph with a checkpointer, as before, and with our new `in_memory_store` variable.

## Basic Usage

In [12]:
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

Memories are namespaced by a `tuple`, which in this specific example will be `(<user_id>, "memories")`. The namespace can be any length and represent anything, does not have to be user specific.

In [14]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

We use the `store.put` method to save memories to our namespace in the store. When we do this, we specify the namespace, as defined above, and a key-value pair for the memory: the key is simply a unique identifier for the memory (`memory_id`) and the value (a dictionary) is the memory itself.

In [16]:
import uuid

memory_id = str(uuid.uuid4())
memory = {"food_preference" : "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

In [24]:
in_memory_store.search(namespace_for_memory)

[Item(namespace=['1', 'memories'], key='c14d3e9f-f7bd-4cb4-9b78-95b17c2894a7', value={'food_preference': 'I like pizza'}, created_at='2025-11-04T08:45:21.491497+00:00', updated_at='2025-11-04T08:45:21.491502+00:00', score=None)]

In [23]:
in_memory_store.search(namespace_for_memory)[-1].dict()

{'namespace': ['1', 'memories'],
 'key': 'c14d3e9f-f7bd-4cb4-9b78-95b17c2894a7',
 'value': {'food_preference': 'I like pizza'},
 'created_at': '2025-11-04T08:45:21.491497+00:00',
 'updated_at': '2025-11-04T08:45:21.491502+00:00',
 'score': None}

## Semantic Search

## Using in LangGraph

We compile the graph with both the checkpointer and the `in_memory_store` as follows.

```python
from langgraph.checkpoint.memory import InMemorySaver

# We need this because we want to enable threads (conversations)
checkpointer = InMemorySaver()

# ... Define the graph ...

# Compile the graph with the checkpointer and store
graph = graph.compile(checkpointer=checkpointer, store=in_memory_store)
```

We can access the `in_memory_store` and the `user_id` in any node by passing `store: BaseStore` and `config: RunnableConfig` as node arguments.

```Python
def update_memory(state: MessagesState, config: RunnableConfig, *, store: BaseStore):

    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "memories")

    # ... Analyze conversation and create a new memory

    # Create a new memory ID
    memory_id = str(uuid.uuid4())

    # We create a new memory
    store.put(namespace, memory_id, {"memory": memory})
```

We can access the memories and use them in our model call.

```Python
def call_model(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "memories")

    # Search based on the most recent message
    memories = store.search(
        namespace,
        query=state["messages"][-1].content,
        limit=3
    )
    info = "\n".join([d.value["memory"] for d in memories])

    # ... Use memories in the model call
```